In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import country_converter as coco
import numpy as np

In [2]:
# Define file location
FILE_LOCATION = "Global_Mobility_Report.csv"

# Load the Citizen CSV as a pandas dataframe, but only selected columns
df = pd.read_csv(FILE_LOCATION, delimiter=",", low_memory=False)
# , usecols=USE_COLS_POPULATION
df['date'] = pd.to_datetime(df['date'], errors='coerce', format= '%Y-%m-%d')

gm = df[df['sub_region_1'].isna()]

gm = gm.drop("sub_region_1", 1)
gm = gm.drop("sub_region_2", 1)
gm = gm.drop("metro_area", 1)
gm = gm.drop("iso_3166_2_code", 1)
gm = gm.drop("census_fips_code", 1)

In [3]:
gm = (gm.groupby([pd.Grouper(key='date', freq='MS'), 'country_region_code', 'country_region'])['retail_and_recreation_percent_change_from_baseline', 
                                                    'grocery_and_pharmacy_percent_change_from_baseline', 'parks_percent_change_from_baseline', 
                                                    'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 
                                                    'residential_percent_change_from_baseline'].mean().reset_index())
gm.head()

/Users/victoriarautenbach/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,date,country_region_code,country_region,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-02-01,AE,United Arab Emirates,-1.066667,2.533333,2.933333,-0.600000,2.333333,0.933333
1,2020-02-01,AF,Afghanistan,-1.200000,5.100000,0.866667,4.966667,2.600000,0.800000
2,2020-02-01,AG,Antigua and Barbuda,-1.333333,0.200000,2.200000,-0.400000,1.200000,1.230769
3,2020-02-01,AO,Angola,-2.733333,-2.333333,1.600000,-4.866667,-6.933333,1.733333
4,2020-02-01,AR,Argentina,-0.666667,-1.400000,-1.066667,1.400000,-1.800000,0.600000


In [4]:
gm['iso3'] = ""

for i, row in gm.iterrows():
    iso2 = row['country_region_code']
    iso3 = coco.convert(names=iso2, to='ISO3')
    gm.at[i,'iso3'] = iso3
    
gm.head()

,date,country_region_code,country_region,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,iso3
0,2020-02-01,AE,United Arab Emirates,-1.066667,2.533333,2.933333,-0.600000,2.333333,0.933333,ARE
1,2020-02-01,AF,Afghanistan,-1.200000,5.100000,0.866667,4.966667,2.600000,0.800000,AFG
2,2020-02-01,AG,Antigua and Barbuda,-1.333333,0.200000,2.200000,-0.400000,1.200000,1.230769,ATG
3,2020-02-01,AO,Angola,-2.733333,-2.333333,1.600000,-4.866667,-6.933333,1.733333,AGO
4,2020-02-01,AR,Argentina,-0.666667,-1.400000,-1.066667,1.400000,-1.800000,0.600000,ARG


In [5]:
gm = gm.drop("country_region_code", 1)
gm = gm.drop("country_region", 1)

df2 = gm.melt(id_vars=["date", "iso3"], 
        var_name="Indicator", 
        value_name="Count")

In [6]:
df2 = df2.rename({'iso3': 'Geography', 'date': 'Date'}, axis=1) 
df2['Date']= df2['Date'].dt.strftime('%b %Y')

retail = df2[df2['Indicator'] == "retail_and_recreation_percent_change_from_baseline"]
grocery = df2[df2['Indicator'] == "grocery_and_pharmacy_percent_change_from_baseline"]
parks = df2[df2['Indicator'] == "parks_percent_change_from_baseline"]
transit = df2[df2['Indicator'] == "transit_stations_percent_change_from_baseline"]
workplaces = df2[df2['Indicator'] == "workplaces_percent_change_from_baseline"]
residential = df2[df2['Indicator'] == "residential_percent_change_from_baseline"]

In [7]:
retail = retail.drop("Indicator", 1)
grocery = grocery.drop("Indicator", 1)
parks = parks.drop("Indicator", 1)
transit = transit.drop("Indicator", 1)
workplaces = workplaces.drop("Indicator", 1)
residential = residential.drop("Indicator", 1)

In [9]:
retail.to_csv(r'retail.csv', index = False, sep=',')
grocery.to_csv(r'grocery.csv', index = False, sep=',')
parks.to_csv(r'parks.csv', index = False, sep=',')
transit.to_csv(r'transit.csv', index = False, sep=',')
workplaces.to_csv(r'workplaces.csv', index = False, sep=',')
residential.to_csv(r'residential.csv', index = False, sep=',')